## Setup

In [7]:
#imports
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import time
import re
import tensorflow as tf
import tensorflow_hub as hub
import os
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import string
import sys
from datetime import datetime

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed
from datetime import datetime

%matplotlib inline

#set working directory
os.chdir("/Users/alexdessouky/Desktop/MIDS/w266")

#load training data
twitter_train = pd.read_excel('./w266_final_project/StanceDataset/train.xlsx')
#train_x = twitter_train[twitter_train['Target'] == 'Hillary Clinton']['Tweet']
#train_y = twitter_train[twitter_train['Target'] == 'Hillary Clinton']['Stance']
train_x = pd.concat([twitter_train[twitter_train['Stance'] == 'AGAINST'].iloc[0:5,]['Tweet'],
                     twitter_train[twitter_train['Stance'] == 'NONE'].iloc[0:5,]['Tweet'],
                      twitter_train[twitter_train['Stance'] == 'FAVOR'].iloc[0:5,]['Tweet']],
                    axis=0)

train_y = np.array([[1,0,0],[1,0,0],[1,0,0],[1,0,0],[1,0,0],
                  [0,1,0],[0,1,0],[0,1,0],[0,1,0],[0,1,0],
                  [0,0,1],[0,0,1],[0,0,1],[0,0,1],[0,0,1]])

#load test data
twitter_test = pd.read_excel('./w266_final_project/StanceDataset/test.xlsx')
#test_x = twitter_test[twitter_test['Target'] == 'Hillary Clinton']['Tweet']
#test_y = twitter_test[twitter_test['Target'] == 'Hillary Clinton']['Stance']
test_x = pd.concat([twitter_test[twitter_test['Stance'] == 'AGAINST'].iloc[0:5,]['Tweet'],
                     twitter_test[twitter_test['Stance'] == 'NONE'].iloc[0:5,]['Tweet'],
                      twitter_test[twitter_test['Stance'] == 'FAVOR'].iloc[0:5,]['Tweet']],
                    axis=0)
test_y = np.array([[1,0,0],[1,0,0],[1,0,0],[1,0,0],[1,0,0],
                  [0,1,0],[0,1,0],[0,1,0],[0,1,0],[0,1,0],
                  [0,0,1],[0,0,1],[0,0,1],[0,0,1],[0,0,1]])

In [23]:
#store the paths to bert & data
bert_path =   '/Users/alexdessouky/Desktop/MIDS/w266/bert' 
data_path = '/Users/alexdessouky/Desktop/MIDS/w266/w266_final_project/StanceDataset'  

now = datetime.now() # current date and time

# make sure that the paths are accessible within the notebook
sys.path.insert(0,local_bert_path)
sys.path.insert(0,data_path)

In [24]:
import optimization
import run_classifier
import tokenization
import run_classifier_with_tfhub

# Tensorflow hub path to BERT module of choice
bert_url = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

# Define maximal length of input 'sentences' (post tokenization).
max_length = 49

## Bert tokenizer

In [13]:
def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(bert_url)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

W1109 10:09:51.304888 140736218923904 deprecation_wrapper.py:119] From /Users/alexdessouky/Desktop/MIDS/w266/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



## Pre-Process Data

In [16]:
#create tokens surrounded by the [CLS] and [SEP] tokens
train_tokens = train_x.apply(lambda x: ['[CLS]'] + tokenizer.tokenize(x) + ['[SEP]'])
test_tokens = test_x.apply(lambda x: ['[CLS]'] + tokenizer.tokenize(x) + ['[SEP]'])

#mask ids (mask out the paddings)
train_mask_ids = train_tokens.apply(lambda x: len(x)*[1])
test_mask_ids = test_tokens.apply(lambda x: len(x)*[1])

train_mask_ids = train_mask_ids.apply(lambda x: np.array(x + (max_length - len(x)) * [0]) if len(x) < max_length else 
                                      np.array(x)).tolist()
test_mask_ids = test_mask_ids.apply(lambda x: np.array(x + (max_length - len(x)) * [0]) if len(x) < max_length else 
                                    np.array(x)).tolist()

#add padding to tokens
train_tokens = train_tokens.apply(lambda x: x + (max_length - len(x)) * ['[PAD]'] if len(x) < max_length else x)
test_tokens = test_tokens.apply(lambda x: x + (max_length - len(x)) * ['[PAD]'] if len(x) < max_length else x)

#test/train sequence vectors
train_sequenceids = train_tokens.apply(lambda x: np.array(max_length*[0])).tolist()
test_sequenceids = test_tokens.apply(lambda x: np.array(max_length*[0])).tolist()

#convert tokens to sentence ids
train_sentenceids = train_tokens.apply(lambda x: np.array(tokenizer.convert_tokens_to_ids(x))).tolist()
test_sentenceids = test_tokens.apply(lambda x: np.array(tokenizer.convert_tokens_to_ids(x))).tolist()

#bert features
bert_train = [np.array(train_sentenceids),np.array(train_mask_ids),np.array(train_sequenceids)]
bert_test = [np.array(test_sentenceids),np.array(test_mask_ids),np.array(test_sequenceids)]

#labels
#stance_labels_train = np.array(twitter_train[twitter_train['Target'] == 'Hillary Clinton']['Stance'].apply(lambda x: 
#                                                                    2 if x == "FAVOR" else 
#                                                                    (1 if x == "NONE" else 0)))

#stance_labels_test = np.array(twitter_test[twitter_test['Target'] == 'Hillary Clinton']['Stance'].apply(lambda x: 
#                                                                    2 if x == "FAVOR" else 
#                                                                    (1 if x == "NONE" else 0)))

            

#PREP LABELS FOR NN
#train_y = np.zeros(shape = (stance_labels_train.shape[0],3))
#train_y[stance_labels_train == 0,0] = 1
#train_y[stance_labels_train == 1,1] = 1
#train_y[stance_labels_train == 2,2] = 1

#test_y = np.zeros(shape = (stance_labels_test.shape[0],3))
#test_y[stance_labels_test == 0,0] = 1
#test_y[stance_labels_test == 1,1] = 1
#test_y[stance_labels_test == 2,2] = 1

In [19]:
train_sentenceids[0]

array([  101,   137, 21359,  1181,  1665,  5082,  1584,  1262,   117,
         108,  9918,  2346,  4121,  1942,  4638,  1708, 22552,  1197,
        1131,  7960,  4044,   116,  1476,  1377, 18931, 24853,   117,
        7155,  4167, 21091,  5796,  1104,  4019,   120,  2887,  1231,
         108,  3096,  5084, 19888,   117,  3576,   108,   189, 18982,
         102,     0,     0,     0])

## Custom layer to create Bert

In [36]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            bert_url,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        trainable_vars = self.bert.variables
        
        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
        
        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
        
        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
        
        # Add non-trainable weights
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)
        
        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [37]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

## Define Model

In [38]:
def bert_model(max_input_length, train_layers, optimizer = tf.keras.optimizers.Adam(learning_rate=1)):
    
    in_id = tf.keras.layers.Input(shape=(max_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_length,), name="segment_ids")
    
    
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_sequence = BertLayer(n_fine_tune_layers=train_layers)(bert_inputs)
    
    
    #dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
    dropout = tf.keras.layers.Dropout(rate=0.3)(bert_sequence)
    
    pred = tf.keras.layers.Dense(3, activation='softmax', name='classification')(dropout)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['categorical_accuracy'])
    
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [39]:
#Start session

sess = tf.Session()

model = bert_model(max_length + 1, train_layers=12, optimizer = 'adam')

# Instantiate variables
initialize_vars(sess)

#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(
    bert_train, 
    train_y,
    #validation_data=[bert_test, test_y],
    epochs=30,
    verbose=1,
    batch_size=32)#,
    #callbacks=[tensorboard]

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 49)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 49)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 49)]         0                                            
__________________________________________________________________________________________________
bert_layer_5 (BertLayer)        (None, 768)          108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]          

In [42]:
test = model.predict(bert_train)

In [43]:
np.argmax(test, axis = 1)

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [ ]:
def confusion_plot(confusion_matrix, target_names):
    # Plot confusion matrix (via imshow)
    plt.imshow(confusion_matrix, interpolation = "nearest", cmap = plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    # Loop through each value of the matrix to add data labels
    width, height = confusion_matrix.shape
    for x in range(width):
        for y in range(height):
            plt.annotate(str(confusion_matrix[x][y]), xy = (y, x), 
                        horizontalalignment = "center",
                        verticalalignment = "center")
    plt.ylabel("True label")
    
def metrics(true_labels, test_probs):
    
    #find predicted labels
    test_predicts = np.argmax(test_probs, axis = 1)
    
    #calculate f1 score
    f1 = f1_score(true_labels, test_predicts, average = 'macro')
    
    print("F1 macro score:", f1)
    
    print(classification_report(y_true = true_labels, 
                                        y_pred = test_predicts,
                                        target_names = ['Against', 'None', 'Favor']))
    
    confuse = confusion_matrix(y_true = true_labels, y_pred = test_predicts)
    
    plt.figure(figsize=(15, 5))
    confusion_plot(confuse, ['Against', 'None', 'Favor'])
    plt.show()